# Алгоритм PCA с авто-подбором числа компонент
---
Функция `pca(X, k=None, threshold=0.95)` реализует стандартный алгоритм **Principal Component Analysis** с возможностью автоматически выбирать число компонент так, чтобы доля объяснённой дисперсии не была меньше порога `threshold`.

### Входные параметры
- $X$ — матрица данных размера $n\times m$ (объекты в строках, признаки в столбцах).
- $k$ — желаемое число компонент; если `None`, то подбирается автоматически.
- `threshold` — порог для выбора $k$ при авто-подборе: минимальная доля объяснённой дисперсии (по умолчанию $0.95$).

### Выход
- $X_{\rm proj}$ — матрица проекций размером $n\times k$.
- $\gamma$ — доля объяснённой дисперсии, суммарно по первым $k$ компонентам.
- $W$ — матрица главных компонент размером $m\times k$ (столбцы — собственные векторы).
- `means` — вектор средних по каждому из $m$ признаков (длина $m$).

---

### Ход алгоритма

1. **Проверка размерности**
   Проверяем, что $n>0$ и $m>0$, иначе выбрасываем `ValueError`.

2. **Центрирование**
   Вычисляем вектор средних по столбцам:
   $$
     \mu_j = \frac{1}{n}\sum_{i=1}^n X_{ij},\quad j=1\ldots m.
   $$
   Центрируем данные:
   $$
     X_{\rm c}[i,j] = X[i,j] - \mu_j.
   $$

3. **Ковариационная матрица**
   Строим матрицу
   $$
     C = \frac{1}{n-1}\,X_{\rm c}^\top X_{\rm c}
   $$
   (или другую реализацию `covariance_matrix`).

4. **Собственные значения и авто-выбор $k$**
   - Находим все собственные значения:
     `eigenvalues = find_eigenvalues(C)`.
   - Если `k is None`, то
     $$
       k = \mathrm{auto\_select\_k}(\text{eigenvalues},\;\text{threshold}),
     $$
     то есть минимальное $k$, для которого
     $$
       \sum_{i=1}^k \lambda_i \Big/ \sum_{i=1}^m \lambda_i \;\ge\;\text{threshold}.
     $$
   - Проверяем, что $1 \le k \le m$.

5. **Собственные векторы**
   Получаем список векторов:
   `eigenvectors = find_eigenvectors(C, eigenvalues)`
   и проверяем, что найдено не менее $k$ векторов.

6. **Сортировка и выбор топ-$k$**
   Формируем пары $(\lambda_i, v_i)$, сортируем по $\lambda$ в порядке убывания и выбираем первые $k$:
   $$
     (\lambda_{(1)}, v_{(1)}),\dots,(\lambda_{(k)}, v_{(k)}).
   $$

7. **Построение матрицы $W$ и проекция**
   Собираем $W\in\mathbb{R}^{m\times k}$, где столбец $j$ — вектор $v_{(j)}$.
   Проецируем центрированные данные:
   $$
     X_{\rm proj} = X_{\rm c}\,W.
   $$

8. **Доля объяснённой дисперсии**
   Считаем
   $$
     \gamma = \frac{\sum_{i=1}^k \lambda_{(i)}}{\sum_{i=1}^m \lambda_i},
   $$
   используя функцию `explained_variance_ratio(eigenvalues, k)`.

9. **Возврат**
   ```python
   return X_proj, gamma, W, means

---

In [1]:
from src.covariance_matrix import covariance_matrix
from src.find_eigenvalues import find_eigenvalues
from src.find_eigenvectors import find_eigenvectors
from src.explained_variance_ratio import explained_variance_ratio
from src.Matrix import Matrix
from src.auto_select_k import auto_select_k

In [2]:
def pca(X: Matrix, k: int = None, threshold: float = 0.95):
    """
    Полный алгоритм PCA с опциональным авто-подбором числа компонент.

    Вход:
      X:       Matrix (n×m) — исходные данные
      k:       int или None — желаемое число компонент
      threshold: float — если k=None, то используется этот порог
                        для auto_select_k(eigenvalues, threshold)
    Выход:
      X_proj: Matrix (n×k) — проекция данных
      gamma:  float        — доля объяснённой дисперсии
      W:      Matrix (m×k) — матрица главных компонент
      means:  list[float]  — вектор средних по колонкам (len=m)
    """
    n, m = X.rows, X.cols
    if n == 0 or m == 0:
        raise ValueError("Пустая матрица X")

    means = [sum(X.data[i][j] for i in range(n)) / n for j in range(m)]
    X_centered = Matrix([[X.data[i][j] - means[j] for j in range(m)]
                         for i in range(n)])

    C = covariance_matrix(X_centered)

    eigenvalues = find_eigenvalues(C)
    if not eigenvalues:
        raise ValueError("Не удалось найти собственные значения")
    if k is None:
        k = auto_select_k(eigenvalues, threshold)
    if not (1 <= k <= m):
        raise ValueError(f"k должно быть в диапазоне [1, {m}], получено {k}")

    eigenvectors = find_eigenvectors(C, eigenvalues)
    if len(eigenvectors) < k:
        raise ValueError(f"Найдено лишь {len(eigenvectors)} векторов, запрошено {k}")

    pairs = list(zip(eigenvalues, eigenvectors))
    pairs.sort(key=lambda x: x[0], reverse=True)
    top_vals, top_vecs = zip(*pairs[:k])

    W = Matrix((m, k))
    for j, vec in enumerate(top_vecs):
        for i in range(m):
            W.data[i][j] = vec.data[i][0]

    X_proj = X_centered @ W

    gamma = explained_variance_ratio(eigenvalues, k)

    return X_proj, gamma, W, means

---